In [2]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
%sql postgresql://postgres@psql:5432/postgres

'Connected: postgres@postgres'

In [9]:
%sql EXPLAIN ANALYZE select * from books where author = 'William Shakespeare';

5 rows affected.


QUERY PLAN
Seq Scan on books (cost=0.00..845.26 rows=2 width=50) (actual time=0.023..2.175 rows=21 loops=1)
Filter: ((author)::text = 'William Shakespeare'::text)
Rows Removed by Filter: 37200
Planning time: 0.112 ms
Execution time: 2.186 ms


In [10]:
%sql EXPLAIN ANALYSE select * from books where author = '%William Shakespeare%';

5 rows affected.


QUERY PLAN
Seq Scan on books (cost=0.00..845.26 rows=2 width=50) (actual time=1.951..1.951 rows=0 loops=1)
Filter: ((author)::text = '%William Shakespeare%'::text)
Rows Removed by Filter: 37221
Planning time: 0.031 ms
Execution time: 1.960 ms


In [13]:
%sql CREATE INDEX book_author_index ON books USING btree("author");

Done.


[]

In [29]:
%sql EXPLAIN ANALYZE select * from books where author like 'William Shakespeare';

7 rows affected.


QUERY PLAN
Bitmap Heap Scan on books (cost=4.31..11.90 rows=2 width=50) (actual time=0.015..0.027 rows=21 loops=1)
Filter: ((author)::text ~~ 'William Shakespeare'::text)
Heap Blocks: exact=13
-> Bitmap Index Scan on book_author_index (cost=0.00..4.30 rows=2 width=0) (actual time=0.010..0.010 rows=21 loops=1)
Index Cond: ((author)::text = 'William Shakespeare'::text)
Planning time: 0.049 ms
Execution time: 0.039 ms


In [28]:
%sql EXPLAIN ANALYSE select * from books where author like '%William Shakespeare%';

5 rows affected.


QUERY PLAN
Seq Scan on books (cost=0.00..845.26 rows=3 width=50) (actual time=0.031..3.081 rows=25 loops=1)
Filter: ((author)::text ~~ '%William Shakespeare%'::text)
Rows Removed by Filter: 37196
Planning time: 0.043 ms
Execution time: 3.092 ms


In [30]:
%%sql Explain analyse
select cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where geodistance(latitude, longitude, 52.38, 11.47) < 50;

15 rows affected.


QUERY PLAN
Hash Join (cost=4617.72..51832.64 rows=488186 width=60) (actual time=23.957..215.241 rows=1254 loops=1)
Hash Cond: (mentions.bookid = books.id)
-> Hash Join (cost=3400.25..44451.93 rows=488186 width=30) (actual time=10.685..201.461 rows=1254 loops=1)
Hash Cond: (mentions.cityid = cities.id)
-> Seq Scan on mentions (cost=0.00..22562.28 rows=1464528 width=8) (actual time=0.015..91.315 rows=1464528 loops=1)
-> Hash (cost=3198.68..3198.68 rows=16126 width=30) (actual time=10.317..10.317 rows=69 loops=1)
Buckets: 16384 Batches: 1 Memory Usage: 133kB
-> Seq Scan on cities (cost=0.00..3198.68 rows=16126 width=30) (actual time=1.958..10.284 rows=69 loops=1)
Filter: ((asin(sqrt(((sin((radians(('52.38'::double precision - latitude)) / '2'::double precision)) ^ '2'::double precision) + (((sin((radians(('11.47'::double precision - longitude)) / '2'::double precision)) ^ '2'::double precision) * cos(radians(latitude))) * '0.610421687981602'::double precision)))) * '7926.3352'::double precision) < '50'::double precision)
Rows Removed by Filter: 48308


```
Note: Testing has shown PostgreSQL's hash indexes to perform no better than B-tree indexes, and the index size and build time for hash indexes is much worse. Furthermore, hash index operations are not presently WAL-logged, so hash indexes may need to be rebuilt with REINDEX after a database crash. For these reasons, hash index use is presently discouraged.

Similarly, R-tree indexes do not seem to have any performance advantages compared to the equivalent operations of GiST indexes. Like hash indexes, they are not WAL-logged and may need reindexing after a database crash.

While the problems with hash indexes may be fixed eventually, it is likely that the R-tree index type will be retired in a future release. Users are encouraged to migrate applications that use R-tree indexes to GiST indexes.
```

In [52]:
%sql create extension earthdistance cascade;

Done.


[]

In [122]:
%sql CREATE INDEX latitude_longitude_index ON cities USING gist (ll_to_earth(latitude, longitude));

Done.


[]

In [119]:
%sql drop index latitude_longitude_index;

Done.


[]

In [123]:
%%sql Explain analyse
select cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) < 50;

15 rows affected.


QUERY PLAN
Hash Join (cost=26750.20..73965.12 rows=488186 width=60) (actual time=345.597..345.597 rows=0 loops=1)
Hash Cond: (mentions.bookid = books.id)
-> Hash Join (cost=25532.73..66584.41 rows=488186 width=30) (actual time=333.936..333.936 rows=0 loops=1)
Hash Cond: (mentions.cityid = cities.id)
-> Seq Scan on mentions (cost=0.00..22562.28 rows=1464528 width=8) (actual time=0.007..0.007 rows=1 loops=1)
-> Hash (cost=25331.15..25331.15 rows=16126 width=30) (actual time=333.920..333.920 rows=0 loops=1)
Buckets: 16384 Batches: 1 Memory Usage: 128kB
-> Seq Scan on cities (cost=0.00..25331.15 rows=16126 width=30) (actual time=333.919..333.919 rows=0 loops=1)
"Filter: (sec_to_gc(cube_distance('(3815617.38379195, 774215.802496072, 5051997.71455748)'::cube, (ll_to_earth(latitude, longitude))::cube)) < '50'::double precision)"
Rows Removed by Filter: 48377


In [155]:
%%sql Explain analyse
select cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where earth_box(ll_to_earth(52.38, 11.47), 50000) @> ll_to_earth(latitude, longitude);

18 rows affected.


QUERY PLAN
Gather (cost=1156.31..23211.26 rows=1453 width=60) (actual time=1.161..208.811 rows=825 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Nested Loop (cost=156.31..22065.96 rows=605 width=60) (actual time=1.489..200.474 rows=275 loops=3)
-> Hash Join (cost=156.02..21879.13 rows=605 width=30) (actual time=1.477..199.733 rows=275 loops=3)
Hash Cond: (mentions.cityid = cities.id)
-> Parallel Seq Scan on mentions (cost=0.00..14019.20 rows=610220 width=8) (actual time=0.006..99.702 rows=488176 loops=3)
-> Hash (cost=155.42..155.42 rows=48 width=30) (actual time=0.123..0.123 rows=38 loops=3)
Buckets: 1024 Batches: 1 Memory Usage: 11kB
-> Bitmap Heap Scan on cities (cost=4.65..155.42 rows=48 width=30) (actual time=0.094..0.115 rows=38 loops=3)


In [149]:
%%sql explain analyse
select distinct on (cities.name) cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where earth_box(ll_to_earth(52.38, 11.47), 50000) @> ll_to_earth(latitude, longitude)
and earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) < 50000;

24 rows affected.


QUERY PLAN
Unique (cost=23035.33..23037.75 rows=16 width=60) (actual time=214.468..214.541 rows=12 loops=1)
-> Sort (cost=23035.33..23036.54 rows=484 width=60) (actual time=214.467..214.494 rows=673 loops=1)
Sort Key: cities.name
Sort Method: quicksort Memory: 120kB
-> Gather (cost=1180.14..23013.74 rows=484 width=60) (actual time=1.723..214.284 rows=673 loops=1)
Workers Planned: 2
Workers Launched: 2
-> Nested Loop (cost=180.14..21965.34 rows=202 width=60) (actual time=6.853..207.154 rows=224 loops=3)
-> Hash Join (cost=179.85..21902.96 rows=202 width=30) (actual time=6.838..203.561 rows=224 loops=3)
Hash Cond: (mentions.cityid = cities.id)


In [188]:
%%sql explain analyse
select distinct on (cities.name) cities.name, cities.latitude, cities.longitude, books.id, books.title,
earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) as distance
from cities
left outer join mentions on (cities.id = mentions.cityid)
left outer join books on (mentions.bookid = books.id)
where earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) < 50000;

19 rows affected.


QUERY PLAN
Unique (cost=385428.51..387869.44 rows=15747 width=68) (actual time=538.746..538.836 rows=25 loops=1)
-> Sort (cost=385428.51..386648.97 rows=488186 width=68) (actual time=538.745..538.787 rows=686 loops=1)
Sort Key: cities.name
Sort Method: quicksort Memory: 125kB
-> Hash Left Join (cost=26750.20..319278.58 rows=488186 width=68) (actual time=339.955..538.511 rows=686 loops=1)
Hash Cond: (mentions.bookid = books.id)
-> Hash Right Join (cost=25532.73..66584.41 rows=488186 width=30) (actual time=330.139..521.889 rows=686 loops=1)
Hash Cond: (mentions.cityid = cities.id)
-> Seq Scan on mentions (cost=0.00..22562.28 rows=1464528 width=8) (actual time=0.008..91.965 rows=1464528 loops=1)
-> Hash (cost=25331.15..25331.15 rows=16126 width=30) (actual time=329.380..329.380 rows=25 loops=1)


In [187]:
%%sql explain analyse
select distinct on (cities.name) cities.name, cities.latitude, cities.longitude, books.id, books.title,
earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) as distance
from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) < 50000;

19 rows affected.


QUERY PLAN
Unique (cost=385428.51..387869.44 rows=15747 width=68) (actual time=527.561..527.635 rows=12 loops=1)
-> Sort (cost=385428.51..386648.97 rows=488186 width=68) (actual time=527.560..527.588 rows=673 loops=1)
Sort Key: cities.name
Sort Method: quicksort Memory: 124kB
-> Hash Join (cost=26750.20..319278.58 rows=488186 width=68) (actual time=337.754..527.386 rows=673 loops=1)
Hash Cond: (mentions.bookid = books.id)
-> Hash Join (cost=25532.73..66584.41 rows=488186 width=30) (actual time=326.853..510.245 rows=673 loops=1)
Hash Cond: (mentions.cityid = cities.id)
-> Seq Scan on mentions (cost=0.00..22562.28 rows=1464528 width=8) (actual time=0.010..86.996 rows=1464528 loops=1)
-> Hash (cost=25331.15..25331.15 rows=16126 width=30) (actual time=326.100..326.100 rows=25 loops=1)


In [118]:
%sql create extension btree_gist;

Done.


[]

In [162]:
%%sql
select distinct on (cities.name) cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where geodistance(latitude, longitude, 52.38, 11.47) < 50;

36 rows affected.


name,latitude,longitude,id,title
Aschersleben,51.75742,11.46084,20136,Encyclopaedia Britannica· 11th Edition· Volume 4· Slice 4
Ballenstedt,51.719,11.23265,25302,Encyclopaedia Britannica· 11th Edition· Volume 13· Slice 1
Bernburg,51.79464,11.7401,29965,History of the Jews· Vol. V (of 6)
Braunschweig,52.26594,10.52673,7825,Germany and the Germans
Calbe,51.90668,11.77478,8478,Encyclopaedia Britannica· 11th Edition· Volume 4· Part 4
Dessau,51.83864,12.24555,9360,English Past and Present
Gardelegen,52.5252,11.39523,26731,The Assault
Genthin,52.40668,12.1592,18539,Tales from the German
Gommern,52.07391,11.82297,27471,The Golden Bough (Third Edition· Vol. 7 of 12)
Halberstadt,51.89562,11.05622,34688,The Project Gutenberg Works of Frederich Schiller in English


In [159]:
%%sql
select distinct on (cities.name) cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where earth_box(ll_to_earth(52.38, 11.47), 50000*1.6) @> ll_to_earth(latitude, longitude);

53 rows affected.


name,latitude,longitude,id,title
Aschersleben,51.75742,11.46084,8929,A Narrative of Some of the Lord's Dealings with George Mueller
Ballenstedt,51.719,11.23265,27931,The Violoncello and Its History
Bernburg,51.79464,11.7401,6533,The Black Death· and The Dancing Mania
Braunlage,51.72651,10.6109,26580,Randolph Caldecott
Braunschweig,52.26594,10.52673,29979,The Rise of the Russian Empire
Calbe,51.90668,11.77478,27471,The Golden Bough (Third Edition· Vol. 7 of 12)
Coswig,51.88618,12.45009,14275,Historical Introductions to the Symbolical Books of the Evangelical Lutheran Church
Dannenberg,53.0967,11.09001,28798,Female Warriors· Vol. II (of 2)
Delitzsch,51.52546,12.34284,30145,The City of God· Volume I
Dessau,51.83864,12.24555,18907,Tales from the German. Volume II.


In [171]:
%%sql
select distinct on (cities.name) cities.name, cities.latitude, cities.longitude, books.id, books.title from cities
join mentions on (cities.id = mentions.cityid)
join books on (mentions.bookid = books.id)
where earth_box(ll_to_earth(52.38, 11.47), 50000*1.6) @> ll_to_earth(latitude, longitude)
and earth_distance(ll_to_earth(52.38, 11.47), ll_to_earth(latitude, longitude)) < 50000*1.6;

35 rows affected.


name,latitude,longitude,id,title
Aschersleben,51.75742,11.46084,20197,Pictures of German Life in the XVIIIth and XIXth Centuries· Vol. I.
Ballenstedt,51.719,11.23265,23448,Letters of Franz Liszt· Volume 2: From Rome to the End
Bernburg,51.79464,11.7401,20828,Encyclopaedia Britannica· 11th Edition· Volume 3· Slice 5
Braunschweig,52.26594,10.52673,25937,Encyclopaedia Britannica· 11th Edition· Volume 14· Slice 2
Calbe,51.90668,11.77478,27471,The Golden Bough (Third Edition· Vol. 7 of 12)
Gardelegen,52.5252,11.39523,27471,The Golden Bough (Third Edition· Vol. 7 of 12)
Genthin,52.40668,12.1592,18539,Tales from the German
Gommern,52.07391,11.82297,29713,The Golden Bough: A Study in Magic and Religion (Third Edition· Vol.
Halberstadt,51.89562,11.05622,24247,Encyclopaedia Britannica· 11th Edition· Volume 12· Slice 7
Havelberg,52.83088,12.07552,3791,The Life of Napoleon I (Volume 2 of 2)


In [172]:
%%sql
select earth_distance(ll_to_earth(52.24856185913086, 11.629449844360352), ll_to_earth(52.38, 11.47))

1 rows affected.


earth_distance
18216.2614555625


In [176]:
%%sql
select * from cities
where name like 'Wolmirstedt';

1 rows affected.


id,name,latitude,longitude,cc,population
2806342,Wolmirstedt,52.24856,11.62945,DE,10860
